# Calculating Saturated Ammonium Sulfate to use in Small Scale Ammonium Sulfate Precipitations

The screen this tool designs is based on Rebecca Oot's Small Scale Ammonium Sulfate Precipitation Screen. Rebbeca is a post-doc in Stephan Wilkens' laboratory, Department of Biochemistry and Molecular Biology, Upstate Medical University, Syracuse, New York, and she initially got the protocol for the screen from Erik Kish-Trier, a former graduate student in Stephan's lab.<br>
A webserver version of this tool can be found [here](http://fomightez.pythonanywhere.com/ammonium_screen/) for those looking for a simpler interface.<br>
(If you are viewing this in a github repo, then it is the non-interactive version of the Notebook. In order to have the fully functional interactive version, you can click [here to open at mybinder.org](http://mybinder.org/repo/fomightez/small_scale_ammonium_sulfate_precipatation_calculator/notebooks/small_scale_ammonium_sulfate_precipitation_calc.ipynb). Alternatively, you can download it and run it somewhere you have set to work with IPython or Jupyter Notebooks or run and edit it in the cloud at [Domino Data Lab](http://www.dominodatalab.com/), [Wakari.io](https://www.wakari.io/), or [SageMath Cloud](https://cloud.sagemath.com) and working in the cloud.)

### Investigator-provided values and options:

Below enter the volume in microliters of protein samples you would like to use in each precipitation mixture. 

(50 ul or 500 ul seem reasonable suggestions depending on what you have available.)

In [17]:
protein_sample_volume_microliters = 500

Below enter in a list with the values separated by commas each of the percent salt conditions at which you'd like to perform the precipitations.

(For example, for 30, 40, 50, and 60% salt as the final concentration, enter 

    percent_salt_values_list = [30, 40, 50, 60]

In [18]:
percent_salt_values_list = [30, 40, 50, 60]

#### That is all that is needed. You can now run this notebook and see the result at the bottom. You can view the individual steps prior to the results.

#  

### Calculating volume of a saturated ammonium sulfate solution to mix with the protein sample volumes:

#### IF YOU JUST WANT THE RESULT BASED ON WHAT YOU ENTERED ABOVE, SKIP TO BOTTOM OF THIS NOTEBOOK.

#### (This is just the behind the scenes stuff.)

In [19]:
# this is the main function of this notebook --> function to calculate needed volume
def volume_sat_ammonium_sulfate_required (protein_sample_volume, percent_salt):
    '''
    given a protein sample volume and a target percent salt (in decimal form) 
    for the mix, it calculates the volume of a 
    saturated ammonium sulfate solution (sat. A.S.) to add.
    
    It is based on the math that:
    
    volume sat. A.S. / (volume sat. A.S. + protein_sample_volume) = percent_salt
    
    For example, for a 500 ul protein sample, to obtain 37.5% final salt concentration,
    that would be worked out as below:
    
    x /( 500 + x) = 0.375
    
    x = 187.5 + 0.375x 
    1.0x - 0.375x = 187.5
    0.625x = 187.5
    x = 187.5/0.625
    x = 300.0 ul
    
    So adding 300 ul of saturated ammonium sulfate solution to a 500 ul protein sample
    would result in the final ammonium sulfate concentration of 37.5%
    
    Example of the function in use:
        volume_sat_ammonium_sulfate_required (500, 0.375)
    
    returns
        300.0
    '''
    volume_ammonium_sulfate_required = (protein_sample_volume * percent_salt)/(1.0 - percent_salt)
    return float(volume_ammonium_sulfate_required)


In [20]:
# helper function to convert percents to decimal value
def convert_percent_to_float(percent):
    '''
    takes a percent as a number (or even string of a number) 
    and converts it to decimal form. 
    
    I made it so if includes a '%' symbol, it still works.
    
    Examples of the function in use:
        convert_percent_to_float(20)
    returns
        0.2
        
         convert_percent_to_float(37.5)
    returns
        0.375
        
        convert_percent_to_float('30%')
    returns
        0.3
        
        convert_percent_to_float('37.5%')
    returns
        0.375
    '''
    if type(percent) == str:
        percent = percent.strip() # strips any exess spaces so % strip will work
        # even if space occurs after due to user text entry
        if '%' in percent:
            return float(percent.strip('%')) / 100.0
        else:
            return float(percent)/ 100.0
    return float(percent)/ 100.0

In [21]:
#performing the actual calculations
volumes_needed_list = []
for percent_salts in percent_salt_values_list:
    percent_salts_deci = convert_percent_to_float(percent_salts)
    volume_needed = volume_sat_ammonium_sulfate_required (protein_sample_volume_microliters, percent_salts_deci)
    volumes_needed_list.append(volume_needed)  

#  

### Formatting of the output:

#### IF YOU JUST WANT THE RESULT BASED ON WHAT YOU ENTERED ABOVE, SKIP TO BOTTOM OF THIS NOTEBOOK.

#### (This is just more behind the scenes stuff.)

In [22]:
def make_markdown_table_of_values(chosen_sample_volume,volumes_list, percents_values_list):
    '''
    takes the sample, volume, the list of needed volumes of ammonium sulfate needed,
    and the percents list and arranges this into an easily readable table in markdown
    '''
    # originally I had the next line brokwn up using '\' but I continue to have to 
    # the problem that when it is in a string, it adds several spaces in middle of
    # string where it occurs. So to make table look good I deleted the '\'
    table_header_text = "| Protein Sample | A.S. (saturated) | % Salt resulting |\n|:-------------- |:----------------:|:----------------:|\n"
    table_text = table_header_text
    for idx, volume_needed in enumerate(volumes_list):
        formatted_volume_needed_str= "%.2f" % volume_needed
        #next if will help allow use of strings as percent values in
        # case entered here as that and will keep code consistent with Django
        # implementation where need handling of possible spaces because
        # in Django version percents are parts of unicode at first which I converted to string
        if type(percents_values_list[idx]) == str:
            percent_value = percents_values_list[idx].strip()
            # also add a way to deal with % in case user added
            if '%' in percent_value:
                percent_value = percent_value.strip('%')
        else:
            percent_value = percents_values_list[idx]
        next_line = "| " + str(chosen_sample_volume) + " ul" \
        + (" "* (12 - len(str(chosen_sample_volume))))+ "| "\
        +  formatted_volume_needed_str + " ul"+ \
        (" "* (14 - len(str(formatted_volume_needed_str))))+ "| " \
        + str(percent_value) + "%" + \
        (" "* (16 - len(str(percent_value))))+"|\n"
        table_text = table_text + next_line
    return table_text


# Originally I just had the uncommented, next code line below as the "THE OUTPUT" cell below. It generated as output the
# raw markdown of the table and was okay looking.
#-------------------
# print make_markdown_table_of_values(protein_sample_volume_microliters,volumes_needed_list,percent_salt_values_list)
#-------------------
# However, I had wanted to display a nicely formatted table as the raw markdown would generate if treated as markdown.
# I had tried several solutions but none seemed to do it satisifactorily, and then when I was looking for how to do something else in a
# Jupyter Notebook, I found the solution at http://stackoverflow.com/questions/32026727/format-output-of-code-cell-with-markdown.
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

#  

#  

#### THE OUTPUT:

In [24]:
print make_markdown_table_of_values(protein_sample_volume_microliters,volumes_needed_list,percent_salt_values_list)

| Protein Sample | A.S. (saturated) | % Salt resulting |
|:-------------- |:----------------:|:----------------:|
| 500 ul         | 214.29 ul        | 30%              |
| 500 ul         | 333.33 ul        | 40%              |
| 500 ul         | 500.00 ul        | 50%              |
| 500 ul         | 750.00 ul        | 60%              |



After selecting it, copy and paste the text in the box above into a text editor. The table is formatted to also work well with markdown editors.  
Or to display here a fancier version of the table as it would appear in markdown editors:

In [25]:
printmd (make_markdown_table_of_values(protein_sample_volume_microliters,volumes_needed_list,percent_salt_values_list))

| Protein Sample | A.S. (saturated) | % Salt resulting |
|:-------------- |:----------------:|:----------------:|
| 500 ul         | 214.29 ul        | 30%              |
| 500 ul         | 333.33 ul        | 40%              |
| 500 ul         | 500.00 ul        | 50%              |
| 500 ul         | 750.00 ul        | 60%              |
